In [ ]:
#only has the operations from the excel

import pandas as pd
import re
from geopy.distance import geodesic
import folium
import numpy as np
from folium.plugins import FeatureGroupSubGroup
from datetime import datetime

# Configurable Parameters
RADIUS_KM = 0.2  # Radius for concurrent operation detection
GRID_SIZE_M = 1000  # Grid size in meters
SINGAPORE_BOUNDS = [(1.2, 103.6), (1.5, 104.1)]  # Bounding box for mainland Singapore

# Load data
file_path = "data_test9.xlsx"
df = pd.read_excel(file_path)

# Strip column names to remove extra spaces
df.columns = df.columns.str.strip()

# Ensure required columns exist
required_cols = {"ref.no", "lat", "lon", "date", "time"}
if not required_cols.issubset(df.columns):
    raise KeyError(f"Missing required columns. Found: {df.columns}")

# Clean and process date and time
df["date"] = df["date"].apply(lambda x: re.split(r"\s+and\s+", str(x))[0])


# Function to handle date and time expansion
def process_date_time(row):
    # Split the 'date' and 'time' columns by semicolon
    date_range = row['date'].split(';')
    time_range = row['time'].split(';')
    
    # Ensure both lists have the same number of entries
    if len(date_range) != len(time_range):
        print(f"Warning: Mismatch in date-time count for ref.no {row['ref.no']}")
        return []
    
    # List to store new rows with assigned date and time
    expanded_rows = []
    
    # Iterate through the date-time pairs
    for i in range(len(date_range)):
        # Handle date formats such as "06 January 2024 to 07 January 2024" or "1 Jan 2024 to 1 Apr 2024"
        date_range_parts = date_range[i].split(' to ')
        
        # Process start and end dates
        start_date_str = date_range_parts[0].strip()
        end_date_str = date_range_parts[-1].strip()  # In case it's a range, use last part
        
        # Try parsing the dates using multiple formats
        start_date = None
        end_date = None
        
        date_formats = ["%d %B %Y", "%d %b %Y", "%d %b %Y", "%d %B %Y"]
        
        # Try parsing with multiple formats
        for date_format in date_formats:
            try:
                start_date = datetime.strptime(start_date_str, date_format)
                end_date = datetime.strptime(end_date_str, date_format)
                break
            except ValueError:
                continue
        
        if not start_date or not end_date:
            print(f"Warning: Invalid date format for ref.no {row['ref.no']} on {date_range[i]}")
            continue
        
        # Split start and end times, handling the LT and range
        time_range_parts = time_range[i].split(' to ')
        start_time_str = time_range_parts[0].strip().replace('LT', '')  # Removing 'LT' for time
        end_time_str = time_range_parts[-1].strip().replace('LT', '')  # Removing 'LT' for time
        
        # Handle cases like "0900LT-1800LT"
        if '-' in start_time_str:
            start_time_str, end_time_str = start_time_str.split('-')
        
        # Convert times into time objects
        try:
            start_time = pd.to_datetime(start_time_str, format="%H%M", errors="coerce").time()
            end_time = pd.to_datetime(end_time_str, format="%H%M", errors="coerce").time()
        except ValueError:
            # Handle cases where time is not correctly parsed, e.g., empty time or invalid format
            print(f"Warning: Invalid time format for ref.no {row['ref.no']} on {time_range[i]}")
            # Assign default time if necessary (e.g., 00:00)
            start_time = end_time = pd.to_datetime('00:00', format="%H:%M").time()
        
        # Create a new row for each date-time pair
        new_row = row.copy()
        new_row['start_date'] = start_date
        new_row['end_date'] = end_date
        new_row['start_time'] = start_time
        new_row['end_time'] = end_time
        
        # Calculate datetime columns
        new_row['start_datetime'] = pd.to_datetime(f"{new_row['start_date']} {new_row['start_time']}")
        new_row['end_datetime'] = pd.to_datetime(f"{new_row['end_date']} {new_row['end_time']}")
        
        # Append the expanded row
        expanded_rows.append(new_row)
    
    return expanded_rows

# Apply the function to expand rows
expanded_data = []
for _, row in df.iterrows():
    expanded_data.extend(process_date_time(row))

# Create a DataFrame from the expanded data
df_expanded = pd.DataFrame(expanded_data)

# Step 1: Identify concurrent operations based on spatial and time overlap
df_expanded["concurrent_ops"] = 0
df_expanded["overlapping_ref_no"] = ""

print("\n=== Drone Operations Overlap Analysis ===\n")

for index, row in df_expanded.iterrows():
    concurrent_count = 0
    overlapping_ref_no = []

    for _, other in df_expanded.iterrows():
        if row["ref.no"] == other["ref.no"]:
            continue  # Skip itself

        # Step 1: Check spatial overlap (within radius)
        distance_km = geodesic((row["lat"], row["lon"]), (other["lat"], other["lon"])).km
        if distance_km > RADIUS_KM:
            continue  # Skip if outside radius

        # Step 2: Check temporal overlap (same date & overlapping time)
        if row["start_datetime"].date() == other["start_datetime"].date() and \
           row["start_datetime"] <= other["end_datetime"] and \
           row["end_datetime"] >= other["start_datetime"]:
            concurrent_count += 1
            overlapping_ref_no.append(other["ref.no"])

    df_expanded.at[index, "concurrent_ops"] = concurrent_count
    df_expanded.at[index, "overlapping_ref_no"] = ", ".join(map(str, overlapping_ref_no)) if overlapping_ref_no else "None"

    if concurrent_count > 0:
        concurrent_count += 1
        print(f"Ref No: {row['ref.no']} overlaps with {overlapping_ref_no} on {row['start_date'].date()} from {row['start_time']} to {row['end_time']}.")

print("\n=== Summary ===")
print(f"Total Drone Operations: {len(df_expanded)}")
print(f"Operations with Overlaps: {df_expanded[df_expanded['concurrent_ops'] > 0].shape[0]}")

# Step 3: Visualization Setup
map_center = [df_expanded["lat"].mean(), df_expanded["lon"].mean()]
m = folium.Map(location=map_center, zoom_start=12)
main_group = folium.FeatureGroup(name="Map Layers").add_to(m)
concurrent_operations_group = FeatureGroupSubGroup(main_group, "Concurrent Operations").add_to(m)

# Function to choose color based on concurrent operations count
def choose_color(count):
    if count == 0:
        return "green"
    elif count == 1:
        return "yellow"
    elif count == 2:
        return "orange"
    else:
        return "red"

# Add concurrent operations circles
for index, row in df_expanded.iterrows():
    color = choose_color(row["concurrent_ops"])
    folium.Circle(
        location=[row["lat"], row["lon"]],
        radius=RADIUS_KM * 1000,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.3,
        popup=f"Ref No: {row['ref.no']}<br>Concurrent Ops: {row['concurrent_ops']}<br>Overlapping with: {row['overlapping_ref_no']}<br>Start: {row['start_datetime']}<br>End: {row['end_datetime']}"
    ).add_to(concurrent_operations_group)

# Add legend
legend_html = '''
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 200px; height: 130px;
            background-color: white; z-index:9999; font-size:14px;
            padding: 10px; border: 1px solid black;">
<b>Legend:</b><br>
<span style="background:green;width:12px;height:12px;display:inline-block;"></span> 1 drone<br>
<span style="background:yellow;width:12px;height:12px;display:inline-block;"></span> 2 drones<br>
<span style="background:orange;width:12px;height:12px;display:inline-block;"></span> 3 drones<br>
<span style="background:red;width:12px;height:12px;display:inline-block;"></span> >3 drones<br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Add Layer Control
folium.LayerControl(collapsed=False).add_to(m)

# Add the map as self-contained HTML
html_content = m.get_root().render()

# Write the HTML to a file (self-contained)
with open("map_with_toggle.html", "w") as file:
    file.write(html_content)

# Save the processed and expanded DataFrame to a new Excel file
output_file_path = "data_with_utilization_analysis.xlsx"
df_expanded.to_excel(output_file_path, index=False)

print("\n✅ Updated file saved as 'data_with_utilization_analysis.xlsx'")
print("✅ Map visualization saved as 'map_with_toggle.html' (includes legend and toggle layers)")

m

ValueError: too many values to unpack (expected 2)

In [3]:
import pandas as pd
import re
from geopy.distance import geodesic
import folium
from folium.plugins import FeatureGroupSubGroup
from datetime import datetime

# Configurable Parameters
RADIUS_KM = 0.2  # Radius for concurrent operation detection
GRID_SIZE_M = 1000  # Grid size in meters
SINGAPORE_BOUNDS = [(1.2, 103.6), (1.5, 104.1)]  # Bounding box for mainland Singapore

# Load data
file_path = "data_test10.xlsx"
df = pd.read_excel(file_path)

# Strip column names to remove extra spaces
df.columns = df.columns.str.strip()

# Ensure required columns exist
required_cols = {"ref.no", "lat", "lon", "date", "time"}
if not required_cols.issubset(df.columns):
    raise KeyError(f"Missing required columns. Found: {df.columns}")

# Clean and process date and time
df["date"] = df["date"].apply(lambda x: re.split(r"\s+and\s+", str(x))[0])


# Function to handle date and time expansion
def process_date_time(row):
    # Split the 'date' and 'time' columns by semicolon
    date_range = row['date'].split(';')
    time_range = row['time'].split(';')
    
    # Ensure both lists have the same number of entries
    if len(date_range) != len(time_range):
        print(f"Warning: Mismatch in date-time count for ref.no {row['ref.no']}")
        return []
    
    # List to store new rows with assigned date and time
    expanded_rows = []
    
    # Iterate through the date-time pairs
    for i in range(len(date_range)):
        # Handle date formats such as "06 January 2024 to 07 January 2024" or "1 Jan 2024 to 1 Apr 2024"
        date_range_parts = date_range[i].split(' to ')
        
        # Process start and end dates
        start_date_str = date_range_parts[0].strip()
        end_date_str = date_range_parts[-1].strip()  # In case it's a range, use last part
        
        # Try parsing the dates using multiple formats
        start_date = None
        end_date = None
        
        date_formats = ["%d %B %Y", "%d %b %Y", "%d %b %Y", "%d %B %Y"]
        
        # Try parsing with multiple formats
        for date_format in date_formats:
            try:
                start_date = datetime.strptime(start_date_str, date_format)
                end_date = datetime.strptime(end_date_str, date_format)
                break
            except ValueError:
                continue
        
        if not start_date or not end_date:
            print(f"Warning: Invalid date format for ref.no {row['ref.no']} on {date_range[i]}")
            continue
        
        # Split start and end times, handling the LT and range
        time_range_parts = time_range[i].split(' to ')
        start_time_str = time_range_parts[0].strip().replace('LT', '')  # Removing 'LT' for time
        end_time_str = time_range_parts[-1].strip().replace('LT', '')  # Removing 'LT' for time
        
        # Handle cases like "0900LT-1800LT"
        if '-' in start_time_str:
            start_time_str, end_time_str = start_time_str.split('-')
        
        # Convert times into time objects
        try:
            start_time = pd.to_datetime(start_time_str, format="%H%M", errors="coerce").time()
            end_time = pd.to_datetime(end_time_str, format="%H%M", errors="coerce").time()
        except ValueError:
            # Handle cases where time is not correctly parsed, e.g., empty time or invalid format
            print(f"Warning: Invalid time format for ref.no {row['ref.no']} on {time_range[i]}")
            # Assign default time if necessary (e.g., 00:00)
            start_time = end_time = pd.to_datetime('00:00', format="%H:%M").time()
        
        # Create a new row for each date-time pair
        new_row = row.copy()
        new_row['start_date'] = start_date
        new_row['end_date'] = end_date
        new_row['start_time'] = start_time
        new_row['end_time'] = end_time
        
        # Calculate datetime columns
        new_row['start_datetime'] = pd.to_datetime(f"{new_row['start_date']} {new_row['start_time']}")
        new_row['end_datetime'] = pd.to_datetime(f"{new_row['end_date']} {new_row['end_time']}")
        
        # Append the expanded row
        expanded_rows.append(new_row)
    
    return expanded_rows

# Apply the function to expand rows
expanded_data = []
for _, row in df.iterrows():
    expanded_data.extend(process_date_time(row))

# Create a DataFrame from the expanded data
df_expanded = pd.DataFrame(expanded_data)

# Step 1: Identify concurrent operations based on spatial and time overlap
df_expanded["concurrent_ops"] = 0
df_expanded["overlapping_ref_no"] = ""

print("\n=== Drone Operations Overlap Analysis ===\n")

for index, row in df_expanded.iterrows():
    concurrent_count = 0
    overlapping_ref_no = []

    for _, other in df_expanded.iterrows():
        if row["ref.no"] == other["ref.no"]:
            continue  # Skip itself

        # Step 1: Check spatial overlap (within radius)
        distance_km = geodesic((row["lat"], row["lon"]), (other["lat"], other["lon"])).km
        if distance_km > RADIUS_KM:
            continue  # Skip if outside radius

        # Step 2: Check temporal overlap (same date & overlapping time)
        if row["start_datetime"].date() == other["start_datetime"].date() and \
           row["start_datetime"] <= other["end_datetime"] and \
           row["end_datetime"] >= other["start_datetime"]:
            concurrent_count += 1
            overlapping_ref_no.append(other["ref.no"])

    df_expanded.at[index, "concurrent_ops"] = concurrent_count
    df_expanded.at[index, "overlapping_ref_no"] = ", ".join(map(str, overlapping_ref_no)) if overlapping_ref_no else "None"

    if concurrent_count > 0:
        concurrent_count += 1
        print(f"Ref No: {row['ref.no']} overlaps with {overlapping_ref_no} on {row['start_date'].date()} from {row['start_time']} to {row['end_time']}.")

print("\n=== Summary ===")
print(f"Total Drone Operations: {len(df_expanded)}")
print(f"Operations with Overlaps: {df_expanded[df_expanded['concurrent_ops'] > 0].shape[0]}")

# Filter for only overlapping operations
df_expanded_overlaps = df_expanded[df_expanded['concurrent_ops'] > 0]

# Step 3: Visualization Setup (Show both all and overlapping operations)
map_center = [df_expanded["lat"].mean(), df_expanded["lon"].mean()]
m = folium.Map(location=map_center, zoom_start=12)
main_group = folium.FeatureGroup(name="Map Layers").add_to(m)

# FeatureGroup for all operations
all_operations_group = FeatureGroupSubGroup(main_group, "All Operations").add_to(m)

# FeatureGroup for overlapping operations
concurrent_operations_group = FeatureGroupSubGroup(main_group, "Concurrent Operations").add_to(m)

# Function to choose color based on concurrent operations count
def choose_color(count):
    if count == 0:
        return "green"
    elif count == 1:
        return "yellow"
    elif count == 2:
        return "orange"
    else:
        return "red"

# Add all operations to the map (non-overlapping and overlapping)
for index, row in df_expanded.iterrows():
    color = choose_color(row["concurrent_ops"])
    folium.Circle(
        location=[row["lat"], row["lon"]],
        radius=RADIUS_KM * 1000,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.3,
        popup=f"Ref No: {row['ref.no']}<br>Concurrent Ops: {row['concurrent_ops']}<br>Overlapping with: {row['overlapping_ref_no']}<br>Start: {row['start_datetime']}<br>End: {row['end_datetime']}"
    ).add_to(all_operations_group)

# Add only overlapping operations to the map
for index, row in df_expanded_overlaps.iterrows():
    color = choose_color(row["concurrent_ops"])
    folium.Circle(
        location=[row["lat"], row["lon"]],
        radius=RADIUS_KM * 1000,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.3,
        popup=f"Ref No: {row['ref.no']}<br>Concurrent Ops: {row['concurrent_ops']}<br>Overlapping with: {row['overlapping_ref_no']}<br>Start: {row['start_datetime']}<br>End: {row['end_datetime']}"
    ).add_to(concurrent_operations_group)

# Add legend
legend_html = '''
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 200px; height: 130px;
            background-color: white; z-index:9999; font-size:14px;
            padding: 10px; border: 1px solid black;">
<b>Legend:</b><br>
<span style="background:green;width:12px;height:12px;display:inline-block;"></span> 1 drone<br>
<span style="background:yellow;width:12px;height:12px;display:inline-block;"></span> 2 drones<br>
<span style="background:orange;width:12px;height:12px;display:inline-block;"></span> 3 drones<br>
<span style="background:red;width:12px;height:12px;display:inline-block;"></span> >3 drones<br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Add Layer Control
folium.LayerControl(collapsed=False).add_to(m)

# Add the map as self-contained HTML
html_content = m.get_root().render()

# Write the HTML to a file (self-contained)
with open("map_with_toggle.html", "w") as file:
    file.write(html_content)

# Save the processed and expanded DataFrame to a new Excel file
output_file_path = "data_with_utilization_analysis_both_layers.xlsx"
df_expanded.to_excel(output_file_path, index=False)

print("\n✅ Updated file saved as 'data_with_utilization_analysis_both_layers.xlsx'")
print("✅ Map visualization saved as 'map_with_toggle.html' (includes legend and toggle layers)")

m



=== Drone Operations Overlap Analysis ===

Ref No: CAAS/AP1/2023/3628 overlaps with ['CAAS/AP1/2023/3768'] on 2023-12-31 from 23:00:00 to 01:00:00.
Ref No: CAAS/AP1/2023/3768 overlaps with ['CAAS/AP1/2023/3628'] on 2023-12-31 from 21:00:00 to 02:00:00.

=== Summary ===
Total Drone Operations: 1493
Operations with Overlaps: 3

✅ Updated file saved as 'data_with_utilization_analysis_both_layers.xlsx'
✅ Map visualization saved as 'map_with_toggle.html' (includes legend and toggle layers)


In [7]:
import pandas as pd
import re
from geopy.distance import geodesic
import folium
from folium.plugins import FeatureGroupSubGroup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image
import time
import os

# Configurable Parameters
RADIUS_KM = 0.2  # Radius for concurrent operation detection
GRID_SIZE_M = 1000  # Grid size in meters
SINGAPORE_BOUNDS = [(1.2, 103.6), (1.5, 104.1)]  # Bounding box for mainland Singapore

# Load data
file_path = "data_test10.xlsx"
df = pd.read_excel(file_path)

# Strip column names to remove extra spaces
df.columns = df.columns.str.strip()

# Ensure required columns exist
required_cols = {"ref.no", "lat", "lon", "date", "time"}
if not required_cols.issubset(df.columns):
    raise KeyError(f"Missing required columns. Found: {df.columns}")

# Clean and process date and time
df["date"] = df["date"].apply(lambda x: re.split(r"\s+and\s+", str(x))[0])

# Process and expand date-time as in your original code...

# Visualize the map as before
map_center = [df_expanded["lat"].mean(), df_expanded["lon"].mean()]
m = folium.Map(location=map_center, zoom_start=12)
main_group = folium.FeatureGroup(name="Map Layers").add_to(m)

# Add the layers and map visualization code as you did previously...
# (all operations, overlapping operations, adding markers, and layer control)

# Save the map as a self-contained HTML
html_content = m.get_root().render()
with open("map_with_toggle.html", "w") as file:
    file.write(html_content)

# Function to save the map as a JPEG using selenium and pillow
def save_map_as_jpeg(html_file_path, output_image_path):
    # Set up Selenium with Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode to not open a window
    chrome_options.add_argument("--window-size=1920x1080")  # Set the window size
    
    # Path to the ChromeDriver (you need to have it installed, e.g., using ChromeDriverManager)
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Open the HTML file with the rendered map
        driver.get(f"file:///{os.path.abspath(html_file_path)}")
        
        # Wait for the map and legend to fully render (increase the sleep time)
        time.sleep(30)  # Give more time for rendering

        # Ensure that the legend and map are fully rendered
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # Scroll to trigger rendering
        time.sleep(1)  # Wait for scrolling to finish
        
        # Take a screenshot
        screenshot_path = "map_screenshot.png"
        driver.save_screenshot(screenshot_path)

        # Open the screenshot using Pillow
        image = Image.open(screenshot_path)
        
        # Save the screenshot as JPEG
        image.convert('RGB').save(output_image_path, "JPEG")
        print(f"Map saved as {output_image_path}")

    finally:
        driver.quit()
        if os.path.exists(screenshot_path):
            os.remove(screenshot_path)  # Clean up the temporary PNG file

# Call the function to save the map as a JPEG
save_map_as_jpeg("map_with_toggle.html", "map_visualization.jpeg")

# Rest of your code...


Map saved as map_visualization.jpeg
